In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 62kB/s 
     |████████████████████████████████| 204kB 45.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=e9f34650e93090563f01f5f4463694f77d0cfcc5269fa5eecc2adc5b45df15d3
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as f
from tqdm import tqdm

In [0]:
# create the session
#conf = SparkConf().set("spark.ui.port", "4050")
conf = SparkConf().set('spark.executor.memory', '10g')

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()


In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-04-19 03:55:10--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.220.9.69, 52.5.232.178, 52.207.7.190, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.220.9.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  18.4MB/s    in 0.7s    

2020-04-19 03:55:11 (18.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.6/json/__init__.py", line 299, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/usr/lib/python3.6/json/__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.6/json/dec

Loading Dataset

In [0]:
#loading data: I uploaded the txt file in colab and read it from there directly
soc = sc.textFile("soc-LiveJournal1Adj.txt")

In [7]:
soc.take(3)

['0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94',
 '1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592',
 '2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878']

In [8]:
soc_1 = soc.map(lambda x: (x.split('\t')[0], x.split('\t')[1]))
soc_1.take(5)

[('0',
  '1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'),
 ('1',
  '0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592'),
 ('2',
  '0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878'),
 ('3', '0,12,41,55,1532,12636,13185,27552,38737'),
 ('4',
  '0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822')]

In [9]:
soc_table = spark.createDataFrame((soc_1), ["UserID", "Friends"])
soc_table.show(4)

+------+--------------------+
|UserID|             Friends|
+------+--------------------+
|     0|1,2,3,4,5,6,7,8,9...|
|     1|0,5,20,135,2409,8...|
|     2|0,117,135,1220,27...|
|     3|0,12,41,55,1532,1...|
+------+--------------------+
only showing top 4 rows



In [10]:
soc_table.cache()

DataFrame[UserID: string, Friends: string]

Algorithm:

In [0]:
#Cross-joined the data to itself, to get pair of UserIDs to be able to easily find out how many mutual friends they have
soc_table.registerTempTable("soc")

query = """Select s1.UserID as UserID_1, s1.Friends as Friends_1, s2.UserID as potential, s2.Friends as mutual_friends
            from soc s1 cross join soc s2"""
soc_crossjoined = spark.sql(query)


In [12]:
soc_crossjoined.head(5)

[Row(UserID_1='0', Friends_1='1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94', potential='0', mutual_friends='1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'),
 Row(UserID_1='0', Friends_1='1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94', potential='1', mutual_friends='0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13

In [0]:
#The function does a cartesian product of the sets of friends for the 2 userIDs to find number of mutual friends they share
def get_reco(row):
  uid = row['UserID_1']
  friends = row['Friends_1'].split(",")
  potential = row['potential']
  mutual = row['mutual_friends'].split(",")
  if(uid != potential):
    if(friends.count(str(potential)) == 0 ):
      mutual_count = len(set(friends)&set(mutual))
    else:
      mutual_count = 0
  else:
    mutual_count = 0
  return (int(uid), int(potential), int(mutual_count))


Reccomendations for a subset of UserIDs

In [0]:
#SUbsetting data for only the specified 10 UserIDs required for the writeup.
#for faster execution and sanity check of results
df = soc_crossjoined.filter(soc_crossjoined.UserID_1.isin([11, 924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]))

In [0]:
recommendations = df.rdd.map(lambda row: get_reco(row))

In [0]:
writeup_results = spark.createDataFrame((recommendations), ["UserID","Reccomendation","Mutual_friends"])

In [17]:
writeup_results.cache()

DataFrame[UserID: bigint, Reccomendation: bigint, Mutual_friends: bigint]

In [0]:
writeup_results.registerTempTable("reccomendation_table")

In [0]:
query = """SELECT * FROM(
            SELECT *, Rank() over(partition by UserID order by Mutual_friends DESC, Reccomendation ASC) as Rank
            FROM reccomendation_table
            WHERE Mutual_friends > 0
            AND UserID != Reccomendation) as r
            WHERE r.Rank <= 10"""

final_output = spark.sql(query)

In [20]:
final_output.cache()

DataFrame[UserID: bigint, Reccomendation: bigint, Mutual_friends: bigint, Rank: int]

In [21]:
writeup_users = final_output.groupby("UserID").agg(f.concat_ws(", ", f.collect_list(final_output.Reccomendation)))
writeup_users = writeup_users.select(col("UserID").alias("UserID"), col("concat_ws(, , collect_list(Reccomendation))").alias("Reccomendations"))
writeup_users.show(11, False)

+------+-------------------------------------------------------------------+
|UserID|Reccomendations                                                    |
+------+-------------------------------------------------------------------+
|9019  |9022, 317, 9023                                                    |
|9020  |9021, 9016, 9017, 9022, 317, 9023                                  |
|9992  |9987, 9989, 35667, 9991                                            |
|9022  |9019, 9020, 9021, 317, 9016, 9017, 9023                            |
|11    |27552, 7785, 27573, 27574, 27589, 27590, 27600, 27617, 27620, 27667|
|8941  |8943, 8944, 8940                                                   |
|9990  |13134, 13478, 13877, 34299, 34485, 34642, 37941                    |
|8942  |8939, 8940, 8943, 8944                                             |
|924   |439, 2409, 6995, 11860, 15416, 43748, 45881                        |
|9021  |9020, 9016, 9017, 9022, 317, 9023                                  |

In [0]:

#writeup_users.write.csv('writeup_users.csv')

#Next part computes the reccomendations for all the UserIDs

Reccomendations for all UserIDs

In [23]:
recommendations_all = soc_crossjoined.rdd.map(lambda row: get_reco(row))
recommendations_all.take(3)

[(0, 0, 0), (0, 1, 0), (0, 2, 0)]

In [24]:
recommendations_all.cache()

PythonRDD[77] at RDD at PythonRDD.scala:53

In [25]:
recommendations_all_final = recommendations_all.filter(lambda row: row[2] > 0)
recommendations_all_final.take(5)

[(0, 135, 2), (0, 2409, 1), (0, 8715, 1), (0, 8932, 1), (0, 10623, 1)]

In [26]:
final_results = spark.createDataFrame((recommendations_all), ["UserID","Reccomendation","Mutual_friends"])
final_results.cache()


DataFrame[UserID: bigint, Reccomendation: bigint, Mutual_friends: bigint]

In [0]:
final_results.registerTempTable("reccomendation_table_final")

query = """SELECT * FROM(
            SELECT *, Rank() over(partition by UserID order by Mutual_friends DESC, Reccomendation ASC) as Rank
            FROM reccomendation_table_final
            WHERE Mutual_friends > 0
            AND UserID != Reccomendation) as r
            WHERE r.Rank <= 10"""

complete_output = spark.sql(query)

In [28]:
complete_output.cache()

DataFrame[UserID: bigint, Reccomendation: bigint, Mutual_friends: bigint, Rank: int]

In [0]:
all_users = complete_output.groupby("UserID").agg(f.concat_ws(", ", f.collect_list(complete_output.Reccomendation)))
all_users = all_users.select(col("UserID").alias("UserID"), col("concat_ws(, , collect_list(Reccomendation))").alias("Reccomendations"))

In [30]:
all_users.cache()


DataFrame[UserID: bigint, Reccomendations: string]

The code chunks after this point take very long to execute and for some reason everytime the hosted session gets disconnected before it completes executing. And so I am unable to store the final reccomendations for all userIDs to a file as requested in the question. :(

In [0]:
all_users.show(5)

In [0]:
#all_users.write.mode('overwrite').json("all_users_reccomendation.json")